In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Flatten, Dense, Conv2D, Rescaling
import os
import time
# import tensorflow_datasets as tfds

In [2]:
strategy = tf.distribute.MirroredStrategy()

2024-05-14 13:58:46.472734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 13:58:49.117957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38218 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2024-05-14 13:58:49.127138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38218 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2024-05-14 13:58:49.128750: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [3]:
# Load data and distribute it

batch_size = 128
img_height = 224
img_width = 224
scratch = os.environ['SCRATCH']
train_dir = os.path.join(scratch,'imagenette/imagenette2/train/')
val_dir = os.path.join(scratch,'imagenette/imagenette2/val/')

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
)

# with strategy.scope():
  # Create the layer(s) under scope.

def dataset_fn(input_context):
  # a tf.data.Dataset
  dataset = train_dataset

  # Custom your batching, sharding, prefetching, etc.
  global_batch_size = 128
  batch_siz = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = dataset.batch(batch_siz, drop_remainder=True)
  dataset = dataset.shard(
      input_context.num_input_pipelines,
      input_context.input_pipeline_id
  )
  return dataset
# with strategy.scope():
    
# distributed_dataset = strategy.distribute_datasets_from_function(dataset_fn)
distributed_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(val_dataset)


Found 9469 files belonging to 10 classes.
Found 3925 files belonging to 10 classes.


2024-05-14 13:58:50.695330: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9469
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}

2024-05-14 13:58:50.702760: W tensorflow/core/grappler/optimizers/data/auto

In [4]:
print(train_dataset.take(0))


<TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [5]:
# Create model

def vgg16():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3), strides=1),
        Conv2D(64, (3, 3), activation='relu', padding='same', strides=1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(128, (3, 3), activation='relu', padding='same', strides=1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(256, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(256, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(256, (3, 3), activation='relu', padding='same', strides=1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        Conv2D(512, (3, 3), activation='relu', padding='same', strides=1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Flatten(),
        Dense(4096, activation='relu'),
        Dense(4096, activation='relu'),
        Dense(1000, activation='softmax')
    ])
    return model

In [6]:
# Create a checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [7]:
with strategy.scope():
  # Set reduction to `NONE` so you can do the reduction yourself.
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True,
      reduction=tf.keras.losses.Reduction.NONE)
  def compute_loss(labels, predictions, model_losses):
    per_example_loss = loss_object(labels, predictions)
    loss = tf.nn.compute_average_loss(per_example_loss)
    if model_losses:
      loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))
    return loss

In [8]:
with strategy.scope():
  test_loss = tf.keras.metrics.Mean(name='test_loss')

  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [9]:
# A model, an optimizer, and a checkpoint must be created under `strategy.scope`.
with strategy.scope():
  model = vgg16()

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

In [10]:
def train_step(inputs):
  images, labels = inputs

  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = compute_loss(labels, predictions, model.losses)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_accuracy.update_state(labels, predictions)
  return loss

def test_step(inputs):
  images, labels = inputs

  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss.update_state(t_loss)
  test_accuracy.update_state(labels, predictions)

In [11]:
EPOCHS = 50

In [12]:
# `run` replicates the provided computation and runs it
# with the distributed input.
@tf.function
def distributed_train_step(dataset_inputs):
  per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

@tf.function
def distributed_test_step(dataset_inputs):
  return strategy.run(test_step, args=(dataset_inputs,))

time_start = time.time()

for epoch in range(EPOCHS):
  # TRAIN LOOP
  total_loss = 0.0
  num_batches = 0
  for x in distributed_dataset:
    total_loss += distributed_train_step(x)
    num_batches += 1
  train_loss = total_loss / num_batches

  # TEST LOOP
  for x in test_dist_dataset:
    distributed_test_step(x)

  if epoch % 2 == 0:
    # checkpoint.save(checkpoint_prefix)
  total_time = time.time() - time_start
  template = ("Time: {} Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, "
              "Test Accuracy: {}")
  print(template.format(total_time, epoch + 1, train_loss,
                         train_accuracy.result() * 100, test_loss.result(),
                         test_accuracy.result() * 100))

  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1


2024-05-14 13:59:28.220087: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2024-05-14 13:59:28.319274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2024-05-14 13:59:28.432360: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2024-05-14 13:59:28.601004: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2024-05-14 13:59:32.177188: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
Time: 36.23625349998474 Epoch 1, Loss: 2.696596622467041, Accuracy: 12.345548629760742, Test Loss: 2.2625107765197754, Test Accuracy: 15.97452163696289
Time: 43.44406843185425 Epoch 2, Loss: 2.1876394748687744, Accuracy: 20.889217376708984, Test Loss: 2.105328321456909, Test Accuracy: 23.69426727294922
Time: 56.63681125640869 Epoch 3, Loss: 1.8827011585235596, Accuracy: 35.230751037597656, Test Loss: 1.868722915649414, Test Accuracy: 37.55413818359375
Time: 63.7699818611145 Epoch 4, Loss: 1.592390537261963, Accuracy: 46.60470962524414, Test Loss: 1.4918851852416992, Test Accuracy: 50.31847381591797
Time: 77.03864622116089 Epoch 5, Loss: 1.3699631690979004, Accuracy: 54.905479431152344, Test Loss: 1.3947936296463013, Test Accuracy: 54.929935455322266
Time: 84.04182529449463 Epoch 6, Loss: 1.1886829137802124, Accuracy: 61.00960922241211, Test Loss: 1.2715445756912231, Test Accuracy: 60.76433181762695
Ti

2024-05-14 14:02:12.659263: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at save_restore_v2_ops.cc:138 : RESOURCE_EXHAUSTED: training_checkpoints/ckpt-8_temp/part-00000-of-00001.data-00000-of-00001.tempstate11898694674970125887; Disk quota exceeded


ResourceExhaustedError: training_checkpoints/ckpt-8_temp/part-00000-of-00001.data-00000-of-00001.tempstate11898694674970125887; Disk quota exceeded [Op:SaveV2]

In [ ]:
gpus = tf.config.list_logical_devices('GPU')
print(gpus)